O objetivo desse notebook é de mostrar a criação da base de movimentações de itens dentro de um cd. Há diversas peculiaridades neste processo porque precisamos de uma serie de informações como:
 - Nome de Filiais
 - Nome de produtos
 - Nome de seções do estoque ( depositos)
 - Nome de usuarios que fizeram essa movimentação
E conciliar tudo isso dentro de uma loop onde pode escolher diversas opções de forma aleatoria.

Importação das bilbiotecas

In [9]:
import logging.config
from datetime import datetime as dt, timedelta
import random
import pandas as pd
import numpy as np
from faker import Faker
import logging
#import psycopg2
import os
# from secrets_psql import HOST,PASSWORD

Preparação dos Logs e inicialização da biblioteca que gera dados randomicos

In [10]:
#Logs Directory
os.makedirs(r'../../Logs',exist_ok=True)

#Log
logging.basicConfig(filename=r"../../Logs/TO_generation.log")

#Faker
fake = Faker()

A partir daqui são a inicialização das variaveis, algumas são listas, algumas são dicionarios para termos mais de uma coluna , por exemplo de item que pode ser atrelado uma categoria.

In [11]:
# Branchs, products, dates e etc
transp_id = 0
branchs = ['SP','RJ','DF','RS','RN','AM','CE','PA','MG','MGR','MGRS','GO','PR','ES']

A lógica dos itens é bastante simples: cada item pertence a uma categoria específica. Já a variável ty_trans, que significa "type of transportation", representa os tipos de movimentação entre seções do estoque, comumente chamadas de depósitos.

Cada movimentação possui uma origem e um destino, ambos pré-definidos de acordo com seu tipo. Dependendo do tipo de movimentação, a origem pode estar restrita a um único depósito ou a um conjunto específico de depósitos — o mesmo se aplica ao destino. No entanto, cada movimentação ocorre entre apenas uma origem

In [12]:
products_by_category = {
    "Móveis": [
        'Cadeira', 'Mesa', 'Sofá', 'Cama', 'Armário', 'Estante', 'Poltrona', 'Guarda-roupa',
        'Luminária', 'Cômoda', 'Mesa de cabeceira', 'Prateleira', 'Rack', 'TV', 'Aparador',
        'Balcão', 'Cortina', 'Tapete', 'Quadro', 'Espelho'
    ],
    "Eletrodomésticos": [
        'Banheira', 'Vaso sanitário', 'Lavabo', 'Chuveiro', 'Espremedor de frutas', 'Cafeteira',
        'Liquidificador', 'Geladeira', 'Fogão', 'Micro-ondas', 'Ar condicionado', 'Ventilador',
        'Exaustor', 'Freezer', 'Batedeira', 'Cortador de grama', 'Secadora de roupas',
        'Máquina de lavar', 'Ferro de passar', 'Aspirador de pó'
    ],
    "Eletrônicos": [
        'Notebook', 'Smartphone', 'Tablet', 'Relógio', 'Fone de ouvido', 'Televisor', 'Projetor',
        'Impressora', 'Computador', 'Câmera fotográfica', 'Roteador', 'Caixa de som',
        'Console de videogame'
    ],
    "Vestuário e Acessórios": [
        'Tênis', 'Bota', 'Sandália', 'Sapato', 'Chinelo', 'Jaqueta', 'Camisa', 'Calça',
        'Short', 'Saia', 'Vestido', 'Blusa', 'Cachecol', 'Boné', 'Óculos', 'Bolsa',
        'Carteira', 'Cinto', 'Mochila', 'Relógio de pulso', 'Colar', 'Anel', 'Brincos', 'Pulseira'
    ]
}

sections = ['001','002','003','004','005','006','007','008','009','010','011','012','013']
users = []
dataList = []

for i in range(50):
    users.append(fake.unique.name())

ty_trans = {
    1:{
        'origin':['001','002','003','004'],
        'destiny':'009'
    },
    2:{
        'origin':['001','002','003','004'],
        'destiny':['005','006','007','008']
    },
    3:{
        'origin':'009',
        'destiny':['010','011','012','013']
    },
    3:{
        'origin':'009',
        'destiny':['001','002','003','004']
    }
}

Note que dependendo da quantidade de dados que botar dentro das branchs, o numero total de linhas pode variar, com essa quantidade, o numero final fica em torno de 2 milhoes.

Aqui, estamos de fato construindo a base de dados. Para evitar a repetição de código, utilizo uma função que recebe todos os parâmetros necessários para gerar as movimentações.

 - HourAdd: Essa variável é usada para definir um "limite de hora". Como a função random.random() retorna um número entre 0 e 1, ao multiplicarmos esse valor pelo número desejado (por exemplo, 12), garantimos que o valor final será, no máximo, 12.
- Branch: A filial que está sendo gerada.
- Day: A função será usada dentro de um loop, e esse valor inteiro será aplicado no timedelta de um datetime.now(), subtraindo dias. (O loop percorre um intervalo de 1 a 365, permitindo adicionar ou subtrair dias a cada iteração.)
- Ttrans: Tipo de movimentação, conforme definido na célula anterior.
- Hourlimit: Caso queiramos que a movimentação ocorra em um período específico, como entre 12h e 16h, configuramos HourAdd para 4 e, em hourlimit, definimos 12. Isso fará com que a movimentação ocorra dentro da faixa horária definida.
- UnitReturn: Em movimentações de retorno, podemos ajustar o número de unidades movimentadas, aumentando ou diminuindo o valor conforme necessário.

A única lógica que precisa ser considerada aqui é sobre os depósitos de origem e destino, que podem ser definidos como um único valor ou uma lista de valores. A função verifica essa condição para garantir que seja escolhido um único depósito de origem e destino para a movimentação.

In [13]:
def processMovi(hourAdd: int,branch: str,day:int,ttrasn:int,transp_id:int, hourlimit:int = 0,unitReturn:int = 1) -> list:
    #Down
    today = (dt.now() - timedelta(days=day)).strftime('%Y-%m-%d')
    hour = int(round((random.random() * hourAdd) + hourlimit, 0))
    minute = int(round(random.random() * 59, 0))
    sec =  int(round(random.random() * 59, 0))

    #We need to discover if origin or destiny is list or unique value
    if type(ty_trans[ttrasn]['origin']) == list:
        dep_ori = int(random.choice(ty_trans[ttrasn]['origin']))
    else:
        dep_ori = int(ty_trans[ttrasn]['origin'])
    
    if type(ty_trans[ttrasn]['destiny']) == list:
        dep_dest = int(random.choice(ty_trans[ttrasn]['destiny']))
    else:
        dep_dest = int(ty_trans[ttrasn]['destiny'])

    category = random.choice(list(products_by_category.keys()))
    product = random.choice(products_by_category[category])
    unit = int(round(random.randint(1,2000) * unitReturn,0))
    user = random.choice(users)

    return [transp_id,branch,today,f'{hour}:{minute}:{sec}',category,product,ttrasn,dep_ori,dep_dest,unit,user]

Agora de fato, é a geração. Um loop para a iteração dos dias, um loop para a iteração das branchs e outro para o numero de ordens de transporte no dia.
Todos esses dados vão ser mandados direto para a função, e note que eu tenho um ID que é incrementado e que cada iteração pode ter até 3 Ordens de Transporte, porque aqui fazemos a ideia de tirar um item do armazenamento, colocar esse item na linha de produção e por fim retornar parcialmente este item.

In [14]:
############### Development of the base
print('Generation the data ...')
for everyBranch in branchs:
    for everyDay in range(-390,0):
        for everyTrasn in range(0, random.randint(25,350)):
            transp_id =transp_id + 1
            dataList.append(processMovi(hourAdd=12,branch=everyBranch,day=everyDay,ttrasn=1,transp_id=transp_id))
            transp_id =transp_id + 1
            dataList.append(processMovi(hourAdd=4,hourlimit=12,branch=everyBranch,day=everyDay,ttrasn=3,transp_id=transp_id))
            
            #return?
            returnUnit = random.choices([True,False],[0.3,0.7],k=1)[0]
            if returnUnit:
                transp_id =transp_id + 1
                dataList.append(processMovi(hourAdd=4,hourlimit=12,branch=everyBranch,day=everyDay,ttrasn=2,transp_id=transp_id,unitReturn= random.random() ))

Generation the data ...


No final, é só colocar a lista de listas dentro de um Dataframe e a base está pronta

In [15]:
df = pd.DataFrame(dataList,columns=['id','branch','date','hour','categoria','produto','move_id','dep_origem','dep_destiny','units','user'])
print(df)

              id branch        date      hour               categoria  \
0              1     SP  2026-06-04  10:23:19             Eletrônicos   
1              2     SP  2026-06-04   15:5:55  Vestuário e Acessórios   
2              3     SP  2026-06-04  13:45:50             Eletrônicos   
3              4     SP  2026-06-04  10:58:17                  Móveis   
4              5     SP  2026-06-04  12:48:42        Eletrodomésticos   
...          ...    ...         ...       ...                     ...   
2363590  2363591     ES  2025-05-11  16:32:26             Eletrônicos   
2363591  2363592     ES  2025-05-11  11:18:42        Eletrodomésticos   
2363592  2363593     ES  2025-05-11  12:20:14  Vestuário e Acessórios   
2363593  2363594     ES  2025-05-11  10:47:56             Eletrônicos   
2363594  2363595     ES  2025-05-11  14:42:43  Vestuário e Acessórios   

                   produto  move_id  dep_origem  dep_destiny  units  \
0                   Tablet        1           1     